In [1]:
'''
Попробуйте использовать полносвязную нейронную сеть для решения задачи классификации. Какое значение метрики accuracy получилось?
'''

'\nПопробуйте использовать полносвязную нейронную сеть для решения задачи классификации. Какое значение метрики accuracy получилось?\n'

In [9]:
import wandb
wandb.login(key="972e1a7e81b595b8a22b3c53552a91707540b820")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
import numpy as np
import cv2
import torch
from albumentations.pytorch import ToTensorV2
import albumentations as A
from scipy.io import loadmat
from torch.utils.data import Dataset
import torchvision
from pathlib import Path
import tarfile
import http.client
import tqdm
import wandb
import time
import matplotlib.pyplot as plt
import torch
import numpy as np


import random
import os
from dataclasses import dataclass

import random
import os
from dataclasses import dataclass

import torch.nn as nn
from torch.utils.data import DataLoader, random_split

# Пойдем учиться.
# В качестве ошибки возьмем BCE
import torch.nn.functional as F
import tqdm
import wandb
from torch.optim import Adam


seed = 0
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)


    
class CifarDataset(Dataset):
    def __init__(self, image_size: int = 32, train=True,):
        self.dataset = torchvision.datasets.CIFAR10(root='./data', train=train, download=True)
        self.transforms = A.Compose(
            [
                # Подгонит под размер (128, 128)
                A.Resize(image_size, image_size),
                A.HorizontalFlip(p=0.5),
                # Пиксели в отрезке [0; 255] - это uint8.
                # Переведем в отрезок [0.0; 1.0] - нейросети будет проще.
                A.ToFloat(max_value=255),
                # Поменяет (H, W, C) -> (C, H, W) и превратит в тензор PyTorch
                ToTensorV2(),
                # Для обогащения: будем переворачивать
            ]
        )
        #assert len(self.imgs) == len(self.labels)

    def __getitem__(self, index) -> tuple[torch.Tensor, int]:
        # Читать будем только одну картинку - и возвращать пару (тензор картинки, ее label)
        image, label = self.dataset[index]
        if self.transforms:
            image = self.transforms(image=np.array(image))["image"]
        return image, label

    def __len__(self):
        return len(self.dataset)
    
    

@dataclass
class Config:
    seed: int = 0

    # Данные
    batch_size: int = 64
    do_shuffle_train: bool = True
    img_size: int = 32
    ratio_train_val_test: tuple[float, float, float] = (0.8, 0.1, 0.1)

    # Модель
    hidden_dim: int = 512
    p_dropout: float = 0.3

    # Обучение
    n_epochs: int = 30
    eval_every: int = 2000
    lr: float = 1e-3


def enable_determinism():
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.use_deterministic_algorithms(True, warn_only=True)


def fix_seeds(seed: int):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.mps.manual_seed(seed)


config = Config()
enable_determinism()
fix_seeds(config.seed)


# Готовим заново датасеты
generator = torch.Generator()
generator.manual_seed(config.seed)


# https://pytorch.org/docs/stable/notes/randomness.html#dataloader
def seed_worker(_):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# Create dataset and dataloaders
train_dataset = CifarDataset(train=True)
test_dataset = CifarDataset(train=False)
# создаем большой датасет на основе класса создания датасета и делим на поддатасеты для обучения. валидации и теста

# Split train dataset into train and validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=config.do_shuffle_train,
    generator=generator,
    drop_last=True,
    # Для скорости будем готовить данные в 4 процессах
    num_workers=4,
    pin_memory=True,
    # Это для воспроизводимости https://pytorch.org/docs/stable/notes/randomness.html#dataloader
    worker_init_fn=seed_worker,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=True,
    num_workers=4,
    pin_memory=True,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=True,
)






# инициализируем и проверяем модель

class SimpleCNNModel(nn.Module):
    def __init__(self, num_classes: int = 10):
        super().__init__()
        self.num_classes = 10

        self.fc = nn.Sequential(
            nn.Conv2d(
                in_channels=3, kernel_size=3, out_channels=128
            ),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(
                in_channels=128, kernel_size=3, out_channels=256
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.3),
            nn.Conv2d(
                in_channels=256, kernel_size=3, out_channels=256
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),
            nn.Dropout(0.3),
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.fc(x)
        return x


model = SimpleCNNModel()
x, y = next(iter(train_loader))
print(x.shape)
print(model(x).shape)




def calc_accuracy(model: nn.Module, loader: DataLoader, device: torch.device):
    count_correct, count_total = 0, 0
    model.eval()
    for img_batch, true_labels in loader:
        img_batch = img_batch.to(device)
        true_labels = true_labels.to(device)
        with torch.no_grad():
            pred_val = model(img_batch)
        # Будем предсказывать самый вероятный класс (т.е. порог 0.5 вероятности).
        # Тогда p > 0.5 будет на положительных логитах, а p < 0.5 - на отрицательных
        pred_labels = torch.argmax(pred_val, dim=1)
        count_correct += (pred_labels == true_labels).sum().item()
        count_total += len(true_labels)
    model.train()
    return count_correct / count_total

def train_loop(
    config: Config,
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    params_subset: list | None = None,
):
    if params_subset is None:
        params_subset = model.parameters()
    optimizer = Adam(params_subset, lr=config.lr)
    model.to(device)

    for epoch in range(config.n_epochs):
        print(f"Epoch #{epoch + 1}/#{config.n_epochs}")
        for i, (img_batch, true_labels) in enumerate(tqdm.tqdm(train_loader)):
            step = epoch * len(train_loader) + i
            img_batch, true_labels = img_batch.to(device), true_labels.to(device)

            optimizer.zero_grad()
            pred_labels = model(img_batch)
            loss = F.cross_entropy(pred_labels, true_labels.long())
            loss.backward()
            optimizer.step()

            wandb.log({"loss": loss.cpu().item()}, step=step)
            if (i + 1) % config.eval_every == 0:
                # Подсчитаем accuracy на всем валидационном датасете
                wandb.log(
                    {"accuracy": calc_accuracy(model, val_loader, device)}, step=step
                )
        # В конце эпохи тоже напечатаем accuracy на val-датасете
        wandb.log({"accuracy": calc_accuracy(model, val_loader, device)}, step=step)
        
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    # Apple Silicon
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("using device", device)

wandb.init(project="dz2", name="simple-fc_dz2", config=config.__dict__)
train_loop(config, model, train_loader, val_loader=val_loader, device=device)
wandb.finish()

Files already downloaded and verified
Files already downloaded and verified
torch.Size([64, 3, 32, 32])
torch.Size([64, 10])
using device cuda


Epoch #1/#30


100%|██████████| 625/625 [00:06<00:00, 99.10it/s] 


Epoch #2/#30


100%|██████████| 625/625 [00:06<00:00, 99.14it/s] 


...

Epoch #29/#30


100%|██████████| 625/625 [00:06<00:00, 96.70it/s] 


Epoch #30/#30


100%|██████████| 625/625 [00:06<00:00, 98.32it/s] 


accuracy,▁▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇████████▇██
loss,██▆▇▆▄▄▅▅▅▃▄▄▅▃▄▃▃▄▂▃▃▅▃▂▁▁▃▅▃▅▂▂▄▃▄▄▂▃▁
accuracy,0.75691
loss,0.66267


In [ ]:
'''
Загрузите предварительно обученную модель из серии ResNet.
Примените технику transfer learning или fine-tuning для решения задачи классификации.
При использовании transfer learning обучите только последний слой классификации, оставив параметры остальных слоев неизменными,
при использовании fine-tuning обучите несколько последних слоев сети.
Загрузите .pt модель и класс модели
'''

In [12]:
import torch
import torch.nn as nn
from torchvision import models


class ModifiedResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # Загрузка предварительно обученной модели
        self.resnet18 = models.resnet18(pretrained=True)

        # Замораживание всех слоев кроме последнего полносвязного
        for param in self.resnet18.parameters():
            param.requires_grad = False

        # Изменение последнего полносвязного слоя
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet18(x)

model = ModifiedResNet18(num_classes=10)

torch.manual_seed(987)
config = Config(lr=1e-3, n_epochs=20)
wandb.init(project="dz2", name="resnet_ft_dz2", config=config.__dict__)
train_loop(config, model, train_loader, val_loader=val_loader, device=device)
wandb.finish()

Epoch #1/#20


100%|██████████| 625/625 [00:06<00:00, 99.56it/s] 


Epoch #2/#20


100%|██████████| 625/625 [00:06<00:00, 101.30it/s]


Epoch #3/#20


...

Epoch #19/#20


100%|██████████| 625/625 [00:06<00:00, 102.25it/s]


Epoch #20/#20


100%|██████████| 625/625 [00:06<00:00, 99.99it/s] 


accuracy,▁▂▄▇▇▇▇▄▆▇▆▇▆▆▅▆█▇▅▄
loss,█▇▇▆▅▆▇█▄▃▃▆▄▆▁▇▅▅▇▅▅▇▆▆▆▅█▅█▄▄▆▆▃▄▄▅▄▄▆
accuracy,0.44712
loss,1.65395


In [ ]:
№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№

In [11]:
import torch.nn as nn
from torchvision.models import resnet34

class ModifiedResNet34(nn.Module):
    def __init__(self, num_classes=10, num_finetune_layers=1):
        super().__init__()
        # Загрузка предварительно обученной модели
        self.base_model = resnet34(pretrained=True)
        
        # Замораживаем все слои
        for param in self.base_model.parameters():
            param.requires_grad = False
            
        # Размораживаем последние num_finetune_layers слоев
        for layer in list(self.base_model.children())[-num_finetune_layers:]:
            for param in layer.parameters():
                param.requires_grad = True
                
        # Заменяем последний полносвязный слой
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)
        torch.nn.init.xavier_uniform_(self.base_model.fc.weight)
        self.base_model.conv2 = nn.Conv2d(in_channels = self.base_model.fc.in_features,
            out_channels = self.base_model.fc.out_features,  kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        torch.nn.init.xavier_uniform_(self.base_model.conv2.weight)
        self.base_model.conv1 = nn.Conv2d(in_channels = self.base_model.fc.in_features,
            out_channels = self.base_model.fc.out_features,  kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        torch.nn.init.xavier_uniform_(self.base_model.conv1.weight)        
    def forward(self, x):
        return self.base_model(x)
    
model = ModifiedResNet34()

# Проверяем имена параметров
for name, param in model.named_parameters():
    print(name)

    
# Получаем параметры fc
fc_params = [v for k, v in model.named_parameters() if 'fc.weight' in k or 'fc.bias' in k or 'conv2.weight' in k or 'conv1.weight' in k]
print(fc_params)


wandb.init(project="dz2", name="resnet_ft_dz2", config=config.__dict__)
train_loop(config, model, train_loader, val_loader=val_loader, device=device)
wandb.finish()

base_model.conv1.weight
base_model.conv1.bias
base_model.bn1.weight
base_model.bn1.bias
base_model.layer1.0.conv1.weight
base_model.layer1.0.bn1.weight
base_model.layer1.0.bn1.bias
base_model.layer1.0.conv2.weight
base_model.layer1.0.bn2.weight
base_model.layer1.0.bn2.bias
base_model.layer1.1.conv1.weight
base_model.layer1.1.bn1.weight
base_model.layer1.1.bn1.bias
base_model.layer1.1.conv2.weight
base_model.layer1.1.bn2.weight
base_model.layer1.1.bn2.bias
base_model.layer1.2.conv1.weight
base_model.layer1.2.bn1.weight
base_model.layer1.2.bn1.bias
base_model.layer1.2.conv2.weight
base_model.layer1.2.bn2.weight
base_model.layer1.2.bn2.bias
base_model.layer2.0.conv1.weight
base_model.layer2.0.bn1.weight
base_model.layer2.0.bn1.bias
base_model.layer2.0.conv2.weight
base_model.layer2.0.bn2.weight
base_model.layer2.0.bn2.bias
base_model.layer2.0.downsample.0.weight
base_model.layer2.0.downsample.1.weight
base_model.layer2.0.downsample.1.bias
base_model.layer2.1.conv1.weight
base_model.layer2

Epoch #1/#30


  0%|          | 0/625 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [10, 512, 3, 3], expected input[64, 3, 32, 32] to have 512 channels, but got 3 channels instead

In [134]:
import wandb
wandb.init(project="dz2", config=config.__dict__, timeout=300)

TypeError: init() got an unexpected keyword argument 'timeout'

In [130]:
wandb.init(project="dz2", name="resnet-finetune", config=config.__dict__)
train_loop(
    config, model, train_loader, val_loader, device=device, params_subset=fc_params
)
wandb.finish()

CommError: Run initialization has timed out after 90.0 sec. 
Please refer to the documentation for additional information: https://docs.wandb.ai/guides/track/tracking-faq#initstarterror-error-communicating-with-wandb-process-

In [131]:
import wandb
wandb.login()

True

In [132]:
wandb.init(project="my_project_name", config=config.__dict__)

KeyboardInterrupt: 

In [7]:
'''
Возьмите предпоследний слой модели и посчитайте косинусное расстояние между картинками одинаковых классов
'''

'\nВозьмите предпоследний слой модели и посчитайте косинусное расстояние между картинками одинаковых классов\n'

In [ ]:
def extract_features(model):
    # Удаление последнего полносвязного слоя из модели
    # Ваш код здесь
    return new_model

# Используйте эту функцию для модификации вашей обученной модели

feature_model = extract_features(resnet)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(# Ваш код здесь)

print("Косинусное расстояние:", cos_sim)